In [1]:
import numpy as np
from collections import defaultdict
from itertools import count

## algorithm

In [2]:
def stream_counter():
    bucket = defaultdict(list)
    timestamp = count(1)
    estimate = None

    while True:
        code = yield estimate
        estimate = None

        # update buckets
        if code is True:
            bucket[1].append(next(timestamp))

            i = 1
            while len(bucket[i]) == 3:
                bucket[2 * i].append(bucket[i][1])
                del bucket[i][:2]
                i *= 2

        elif code is False:
            next(timestamp)

        # estimate count
        elif isinstance(code, int):
            counts = [i for i in bucket for t in bucket[i] if code < t] or [0]
            estimate = sum(counts) - counts[-1] // 2
        
        # debug
        elif code == 'debug':
            for i in bucket:
                print(i, bucket[i])

## run

In [3]:
n = 10 ** 6
ctr = stream_counter()
next(ctr)
for i in range(n):
    ctr.send(np.random.rand() >= .5)

for i in np.linspace(.99, 0, 5):
    k = int(i * n)
    print(f'last {n - k} bits: {ctr.send(k)}')

last 10000 bits: 6932
last 257501 bits: 140052
last 505000 bits: 303892
last 752500 bits: 434964
last 1000000 bits: 434964


In [4]:
ctr.send('debug')

1 [999999, 1000000]
2 [999998]
4 [999989, 999995]
8 [999984]
16 [999944, 999970]
32 [999910]
64 [999850]
128 [999732]
256 [998922, 999481]
512 [997478, 998417]
1024 [996449]
2048 [994456]
4096 [990348]
8192 [965868, 982223]
16384 [949681]
32768 [851510, 916898]
65536 [655300, 785917]
131072 [261942, 524040]
